## Using CCD algorithem for solving Inverse kinematics
- Used the cyclic coordinate descent for solving the inverse kineamtics for a 5 DOF arm 
- It generate the trajectory according to the minimum jerk trajectory like the previous version (8), this version is 9
- The point is choosen randomly in space we can change them to choose specific points
- Readded forward kinematics to the get the positions of the joints and the end effector 
- Every thing else stayed nearly the same with minor adjustments

## Next steps
- [] Try to move from Kinematics to dynamics, to account for forces, masses, energy and inertia
- [] Adding another trajectory generation type
- [] Try to find a way to optimize the CCD algo 
- [] Add the analysis plots
- [] 

In [ ]:
import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import time
from scipy.spatial.transform import Rotation

class Arm3D:
    """3D arm inverse kineamtics solver with a 5 DOF model using CCD algorithem IK"""

    def __init__(self):
        # Changed the lengths and added the hand length, all in meters
        self.upper_arm_length = 0.3 
        self.forearm_length = 0.25
        self.hand_length = 0.15 

        # Uptaded the joint limits to the new model, all in radians
        # For Opensim model FullBodyModel_MuscleActuatedLowerLimb_TorqueActuatedUpperBody
        self.joint_limits = {
            'arm_flex_r': (-np.radians(90), np.radians(90)),  # Shoulder elevation
            'arm_add_r': (-np.radians(120), np.radians(90)),  # Shoulder rotation
            'arm_rot_r': (-np.radians(90), np.radians(90)),  # Shoulder abduction
            'elbow_flex_r': (0, np.radians(150)),    # Elbow flexion
            'pro_sup_r': (np.radians(0), np.radians(90))     # Wrist flexion
        }

        # Joint hierarchy for CCD IK, from the base to the end effector
        self.joint_hierarchy = ['arm_flex_r', 'arm_add_r', 'arm_rot_r', 'elbow_flex_r', 'pro_sup_r']

    def forward_kinematics(self, joint_angles):
        """Claculate the end effector position given the joint angles."""
        shoulder_flex = joint_angles['arm_flex_r']
        shoulder_add = joint_angles['arm_add_r']
        shoulder_rot = joint_angles['arm_rot_r']
        elbow_flex = joint_angles['elbow_flex_r']
        forearm_rot = joint_angles['pro_sup_r']

        # Origin shoulder position
        shoulder_pos = np.array([0, 0, 0])

        # Shoulder rotation matrix
        R_shoulder = (Rotation.from_euler('x', shoulder_flex) *
                      Rotation.from_euler('y', shoulder_add) *
                        Rotation.from_euler('z', shoulder_rot)).as_matrix()
        
        # Elbow position
        elbow_pos = shoulder_pos + R_shoulder @ np.array([self.upper_arm_length, 0, 0])

        # Elbow rotation matrix
        R_elbow = (Rotation.from_euler('x', elbow_flex) * Rotation.from_euler('x', forearm_rot)).as_matrix()

        # Wrist position
        wrist_pos = elbow_pos + R_shoulder @ R_elbow @ np.array([self.forearm_length, 0, 0])

        # Hand position (end effector)
        hand_pos = wrist_pos + R_shoulder @ R_elbow @ np.array([self.hand_length, 0, 0])

        return hand_pos, elbow_pos, wrist_pos, shoulder_pos
    
    def get_joint_position(self, joint_angles, joint_name):
        """Get the position of a specific joint."""
        hand_pos, elbow_pos, wrist_pos, shoulder_pos = self.forward_kinematics(joint_angles)
        if joint_name == 'arm_flex_r' or joint_name == 'arm_add_r' or joint_name == 'arm_rot_r':
            return shoulder_pos
        elif joint_name == 'elbow_flex_r':
            return elbow_pos
        elif joint_name == 'pro_sup_r':
            return wrist_pos
        return hand_pos  # End effector
        
    def inverse_kinematics(self, target_pos, initial_angles=None, max_iterations=100, tolerance=1e-3):
        """Perform CCD IK to reach the target position."""
        if initial_angles is None:
            # Start from a neutral position
            # I should change these to start from a the inital reaching position
            initial_angles = {
                'arm_flex_r': 0.0,
                'arm_add_r': 0.0,
                'arm_rot_r': 0.0,
                'elbow_flex_r': np.radians(90),
                'pro_sup_r': 0.0
            }

        angles = initial_angles.copy()
        
        for iteration in range(max_iterations):
            # Perform CCD from the end effector to the base
            for joint_name in reversed(self.joint_hierarchy):
                # Get current end effector position
                current_ee_pos, _,_,_ = self.forward_kinematics(angles)

                # Check convergence
                if np.linalg.norm(target_pos - current_ee_pos) < tolerance:
                    return angles
                
                # Get joint position
                joint_pos = self.get_joint_position(angles, joint_name)

                # Vectors from joint to end effector and joint to target
                current_vec = current_ee_pos - joint_pos
                target_vec = target_pos - joint_pos

                # Normalize vectors
                current_vec_norm = current_vec / np.linalg.norm(current_vec)
                target_vec_norm = target_vec / np.linalg.norm(target_vec)

                # Calculate rotation axis and angle
                rotation_axis = np.cross(current_vec_norm, target_vec_norm)
                axis_norm = np.linalg.norm(rotation_axis)

                if axis_norm > 1e-6:
                    rotation_axis /= axis_norm
                    dot_product = np.clip(np.dot(current_vec_norm, target_vec_norm), -1.0, 1.0)
                    rotation_angle = np.arccos(dot_product)

                    # Apply damping for stability
                    damping = 0.7
                    rotation_angle *= damping

                    # Apply the rotation to the joint
                    angles = self.apply_joint_rotation(angles, joint_name, rotation_axis, rotation_angle)

                    # Final enforcement of convergence checking
                    current_ee_pos, _,_,_ = self.forward_kinematics(angles)
                    if np.linalg.norm(target_pos - current_ee_pos) < tolerance:
                        break

        return angles  # Return the best effort angles after max iterations
    
    def apply_joint_rotation(self, angles, joint_name, rotation_axis, rotation_angle):
        """Apply rotation to a specific joint"""
        new_angles = angles.copy()

        # Map rotation axis to joint angle changes
        if joint_name == 'arm_flex_r':
            # Shoulder flexion/extension around x-axis
            if abs(rotation_axis[0]) > 0.5:
                new_angles[joint_name] += rotation_angle * np.sign(rotation_axis[0])

        elif joint_name == 'arm_add_r':
            # Shoulder abduction/adduction around y-axis
            if abs(rotation_axis[1]) > 0.5:
                new_angles[joint_name] += rotation_angle * np.sign(rotation_axis[1])
        elif joint_name == 'arm_rot_r':
            # Shoulder internal/external rotation around z-axis
            if abs(rotation_axis[2]) > 0.5:
                new_angles[joint_name] += rotation_angle * np.sign(rotation_axis[2])
        elif joint_name == 'elbow_flex_r':
            # Elbow flexion/extension around y-axis
            if abs(rotation_axis[0]) > 0.5:
                new_angles[joint_name] += rotation_angle * np.sign(rotation_axis[1])
        elif joint_name == 'pro_sup_r':
            # Forearm pronation/supination around x-axis
            if abs(rotation_axis[0]) > 0.5:
                new_angles[joint_name] += rotation_angle * np.sign(rotation_axis[0])
        else:
            raise ValueError(f"Unknown joint name: {joint_name}")
        
        # Enforce joint limits
        new_angles[joint_name] = np.clip(new_angles[joint_name], *self.joint_limits[joint_name])

        return new_angles
    
class TrajectoryGenerator:
    """
    Generates smooth trajectories between points.
    Add another trajectory type to the code
    """
    
    @staticmethod
    def minimum_jerk_profile(t_norm):
        """Minimum jerk velocity profile: 10t³ - 15t⁴ + 6t⁵"""
        return 10 * t_norm**3 - 15 * t_norm**4 + 6 * t_norm**5
    
    def generate_trajectory(self, start_pos, end_pos, duration, num_points, trajectory_type='minimum_jerk'):
        """Generate smooth trajectory between start and end positions."""
        
        # Time vector
        t = np.linspace(0, duration, num_points)
        t_norm = t / duration  # Normalized time [0, 1]
        
        # Generate position profile based on type
        if trajectory_type == 'minimum_jerk':
            s = self.minimum_jerk_profile(t_norm)
        elif trajectory_type == 'linear':
            s = t_norm
        elif trajectory_type == 'bell_shaped':
            s = 0.5 * (1 - np.cos(np.pi * t_norm))
        else:
            s = t_norm  # Default to linear
        
        # Interpolate between start and end positions
        start_pos = np.array(start_pos)
        end_pos = np.array(end_pos)
        
        trajectory = []
        for i in range(num_points):
            pos = start_pos + s[i] * (end_pos - start_pos)
            trajectory.append(pos)
        
        return t, trajectory

class OpenSimReachingSystem:
    """3D reaching system with CCD IK"""
    def __init__(self, model_path:str):
        self.model_path = model_path
        self.model = osim.Model(model_path)
        self.state = self.model.initSystem()
        self.trajectory_generator = TrajectoryGenerator()
        self.arm_3d = Arm3D()
        self._setup_arm_parameters()
    
    def _setup_arm_parameters(self):
        """Set up arm parameters and verify coordinate names."""
        self.joint_names = ['arm_flex_r', 'arm_add_r', 'arm_rot_r', 'elbow_flex_r', 'pro_sup_r']

        coord_set = self.model.getCoordinateSet()
        self.valid_coords = True
        for joint_name in self.joint_names:
            try:
                coord_set.get(joint_name)
            except:
                print("❌ Error: Coordinate '{joint_name}' names don't match model")
                print(f"Available coordinates:")
                for i in range(coord_set.getSize()):
                    print(f"- {coord_set.get(i).getName()}")
                self.valid_coords = False
                break
    
    def set_arm_pose(self, joint_angles):
        """Set the arm pose in the OpenSim model with ther limits."""
        if not self.valid_coords:
            return False
        
        try:
            coord_set = self.model.getCoordinateSet()
            for joint_name, angle in joint_angles.items():
                coord = coord_set.get(joint_name)
                # Enforce joint limits
                angle_clipped = np.clip(angle, *self.arm_3d.joint_limits[joint_name])
                coord.setValue(self.state, angle_clipped)
            
            self.model.realizePosition(self.state)
            return True
        except Exception as e:
            print(f"❌ Error setting pose: {e}")
            return False
    
    def create_motion_file(self, time_points, joint_angles_history, filename):
        """Create OpenSim motion file."""
        if not self.valid_coords:
            return False
        
        with open(filename, 'w') as f:
            # Write header
            f.write("name reaching_movement_3d\n")
            f.write("datacolumns {}\n".format(len(self.joint_names) + 1))
            f.write("datarows {}\n".format(len(time_points)))
            f.write("range {} {}\n".format(time_points[0], time_points[-1]))
            f.write("endheader\n")
            
            # Write column labels
            f.write("time\t{}\t{}\t{}\t{}\t{}\n".format(*self.joint_names))
            
            # Write data (convert angles to degrees)
            for t, angles in zip(time_points, joint_angles_history):
                angles_values = [np.degrees(angles[name]) for name in self.joint_names]
                f.write("{:.6f}\t{:.6f}\t{:.6f}\t{:.6f}\t{:.6f}\t{:.6f}\n".format(t, *angles_values))
        
        print(f"✅ Motion file created: {filename}")
        return True
    
    def generate_reaching_movement(self, start_angles, target_pos, duration=2.0, num_points=50):
        """Generate complete 3D reaching movement using CCD IK."""
        # Start position for the trajectory generation
        start_pos, _, _, _ = self.arm_3d.forward_kinematics(start_angles)
        time_points, cartesian_traj = self.trajectory_generator.generate_trajectory(
            start_pos, target_pos, duration, num_points, 'minimum_jerk'
        )

        # Convert each cartesian point to joint angles using CCD IK
        joint_angles_history = []
        positions = []

        current_angles = start_angles.copy()

        for i, target_point in enumerate(cartesian_traj):
            # Perform IK to get joint angles for the target point
            """ik_result = self.arm_3d.inverse_kinematics(target_point, initial_angles=current_angles)
            joint_angles_history.append(ik_result)
            positions.append(target_point)
            current_angles = ik_result"""
            current_angles = self.arm_3d.inverse_kinematics(target_point, current_angles)
            joint_angles_history.append(current_angles.copy())

            # Calculate the actual end effector position for visualization
            achieved_pos, _, _, _ = self.arm_3d.forward_kinematics(current_angles)
            positions.append(achieved_pos)

            print(f"Point {i+1}/{len(cartesian_traj)}: Error = {np.linalg.norm(target_point - achieved_pos):.4f}m")

        return time_points, joint_angles_history, np.array(positions)

def generate_3d_reaching_movement(model_path, output_file, num_points, duration):
    """Generate a reaching movement using CCD."""
    system = OpenSimReachingSystem(model_path)

    if not system.valid_coords:
        return False
    
    # Define a starting pose
    start_angles = {
        'arm_flex_r': np.radians(20),
        'arm_add_r': np.radians(0),
        'arm_rot_r': np.radians(0),
        'elbow_flex_r': np.radians(90),
        'pro_sup_r': np.radians(0)
    }

    # Generate a reachable target
    target_pos = np.array([np.random.uniform(0.1, 0.5),
        np.random.uniform(-0.3, 0.3),
        np.random.uniform(-0.2, 0.4)])
    
    print(f"3D Target Position: {target_pos}")

    # Generate reaching movement using CCD IK
    time_points, joint_angles_history, positions = system.generate_reaching_movement(
        start_angles, target_pos, duration, num_points
    )

    # Creating motion file
    system.create_motion_file(time_points, joint_angles_history, output_file)

    """
    I have to add the function for plotting the analysis plots
    # Generate analysis plots
    system.plot_trajectory_analysis(time_points, positions, joint_angles_history)
    """

    return True

if __name__ == "__main__":
    model_path = "W:\Models\FullBodyModel-4.0\Rajagopal2015.osim"  # Replace with the model of your choice
    output_file1 = "3d_reach_CCD1.mot"
    
    print("Generating 3D reaching movement using CCD IK...")

    if generate_3d_reaching_movement(model_path, output_file1,100, 5.0):
        print("\n🎉 Successfully generated 3D motion file!")
        print("To use in OpenSim:")
        print(f"1. Load your model '{model_path}'")
        print(f"2. Load the motion file '{output_file1}'")
    else:
        print("\n❌ Failed to generate valid motion file")

    output_file2 = "3d_reach_CCD2.mot"
    if generate_3d_reaching_movement(model_path, output_file2, 1000, 5.0):
        print("\n🎉 Successfully generated 3D motion file!")
        print("To use in OpenSim:")
        print(f"1. Load your model '{model_path}'")
        print(f"2. Load the motion file '{output_file2}'")
    else:
        print("\n❌ Failed to generate valid motion file")



                

                


